In [1]:
%matplotlib notebook

In [2]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from apiconfig import g_key

In [3]:
file = "../Resources/output_data_cities.csv"
cities_df = pd.read_csv(file)
cities_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)
weight = cities_df["Humidity"].astype(float)

# Add Heatmap layer to map
fig = gmaps.figure(center=(50.0, -35.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Create new DataFrame fitting weather criteria
cities_filter = cities_df[cities_df['Max Temp'] < 85] 
cities_filter = cities_filter[cities_filter['Max Temp'] > 75] 


cities_filter = cities_filter[cities_filter['Wind Speed'] < 7] 

cities_filter = cities_filter[cities_filter['Cloudiness'] < 5]

cities_filter

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07


In [6]:
hotel_df = cities_filter
hotel_df['Hotel Name'] = ""
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,
268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73,
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    #set base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        print(f'adding data for row {index}')
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

adding data for row 88
adding data for row 125
adding data for row 139
Missing field/result... skipping.
adding data for row 176
adding data for row 268
Missing field/result... skipping.
adding data for row 494
Missing field/result... skipping.


In [8]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,Orlanova Hotel
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,"Hotel Marchica Lagoon Resort, Nador Morocco"
268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73,
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07,


In [9]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [11]:
# Create a marker_layer using the hotel info list to fill the info box
fig = gmaps.figure(center=(50.0, -35.0), zoom_level=2)
markers = gmaps.marker_layer(locations,
    info_box_content=(hotel_info))
fig.add_layer(markers)

fig


Figure(layout=FigureLayout(height='420px'))